# Zanesení bibliografických dat do bipartitního grafu

V tomto notebooku budeme pracovat s Bibliografií českého literárního exilu, která obsahuje záznamy o knihách a statích s literární tématikou, vydaných a publikovaných v českých exilových nakladatelstvích a časopisech. Nás budou zajímat pouze záznamy o statích, u kterých jsou uvedeni autor a časopis, ve kterém stať vyšla. Ty najdeme v polích `100 $a` (respektive `100 $7`) a `773 $t`. Notebook lze použít na všechny bibliografie. <br>
Ukážeme si, jak z dat získat jméno časopisu, jak data o autorech a časopisech následně zpracovat. Nakonec z nich vytvoříme strukturu bipartitního grafu, kterou vykreslíme pomocí knihovny `matplotlib`.   <br>
Tento notebook je určený jak pro začátečníky, tak pro ty, kteří se chtějí seznámit se zpracováním dat v Pythonu.

## Požadavky

K práci s notebookem je potřeba mít vytvořený CSV soubor z notebooku Explore Data.<br>

## Předpoklady

Tento notebook nepředpokládá hluboké znalosti Pythonu, ale základní znalost programování bude užitečná.<br> 

## Struktura notebooku

Notebook je rozdělen do několika částí:

0. **Příprava**: Přidáme potřebné knihovny, které budeme používat ke zpracování marcového souboru. 

1. **Načtení z CSV**: Ukážeme si, jak načíst naše data uložená v CSV.

2. **Extrahování a čištění dat**: Data z jednotlivých sloupců vyextrahujeme a očistíme.

3. **Zpracování**: Z našich očištěných dat získáme nejčastější autory a časopisy, do kterých přispívali. 

4. **Vytvoření struktury grafu**: Ukážeme si, jak pomocí knihovny networkx vytvoříme strukturu grafu, jak so ní přidáme uzle a hrany. Nakonec ji vykreslíme.

## Další zdroje

- [LearnPython.org](https://www.learnpython.org/): Tento online kurz nabízí výuku jazyka Python pro začátečníky i pokročilé. Může být užitečným zdrojem pro ty, kteří chtějí rozšířit své znalosti Pythonu.
- [W3Schools.com/Python](https://www.w3schools.com/python/): Obsáhlý tutoriál, který provází i některými oblíbenými knihovnamy Pythonu. 


### 0. Příprava 
Jako první si musíme nainstalovat knihovny, se kterými budeme pracovat. Knihovny jsou balíčky funkcí, které nejsou součástí základu jazyka python. <br>
Knihovny nainstalujeme pomocí příkazu `%pip install <jmeno_knihovny>` . Pak je do našeho notebooku přidáme pomocí příkazu `import <jmeno_knihovny> (as alias)`. K funkcím knihovny se pak přistupuje `jmeno_knihovny.jmeno_funkce` <br> 
Pokud z knihovny chceme využít pouze jednu funkci, přidáme ji pomocí `from <jmeno_knihovny> import <jmeno_funkce>`

In [ ]:
# Install libraries
%pip install matplotlib
%pip install networkx
%pip install numpy
%pip install pandas

# Add libraries
from collections import Counter
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import re

### 1. Načtení z CSV

Nejprve pomocí knihovny pandas načteme naše uložená CSV data do datové struktury DataFrame (která je podobná např. excelovské tabulce). Řádky v DataFramu reprezentují jednotlivé záznamy, sloupce pak jeden typ (např. jmeno autora).
Některá pole a podpole mohou opakovat, ty jsou  v CSV spojené středníkem. V DataFramu pak funkcí `split()` hodnoty rozpojíme a převede do listu (seznamu). Pokud zrovna na daném místě žádná hodnota není, přidáme prázdný list.

In [ ]:
# Select base
base = 'cle'

# Path to our data
csv_data = 'data/csv/out_{base}.csv'.format(base = base)

# Load data
df = pd.read_csv(csv_data, delimiter=',')

print("Data loaded to DataFrame df.")

# Iterate through column in the DataFrame
for column in df.columns:
    # There is only one publication date, so we don't need to split this column
    if column != 'year': 

        # Split joined values into a list 
        df[column] = df[column].apply(lambda x: x.split(';') if isinstance(x, str)  else [])


Podíváme, jak naše data v tabulce vypadají. Nejprve si vypíšeme, kolik záznamů obsahuje informaci o časopisu. To jsou záznamy o statích, které nás budou zajímat. Zbylé budeme považvat za záznamy o knihách. Pak si vypíšeme prvních 5 a posledních 5 položek v DataFramu. <br>

Nejprve si pomocí lambda funkce najdeme řádky, které obsahují informaci o časopisu. K nim přiřadíme 1 a všechny sečteme. Tím získáme celkový počet záznamů o statích. <br>
Od celkového počtu záznamů pak počet záznamů o statích odečteme, čímž získáme počet záznamů o knihách. Do našeho DataFramu `df` si nakonec uložíme jen záznamy o statích. 

###### Pokud bychom chtěli přesná data, v marcovém záznamu je kolonka LDR (leader), která nese informaci o typu záznamu. 

In [ ]:
# Count nonempty rows 
magazines_counts = df['magazine'].apply(lambda x: 1 if len(x) > 0 else 0)

sum_magazines_counts = magazines_counts.sum()

print("Number of article records: ", sum_magazines_counts)

sum_books_counts = len(df) - sum_magazines_counts

print("Number of books records: ", sum_books_counts)

# Filter book record 
df = df[df['magazine'].apply(lambda x: len(x) > 0)]

In [ ]:
# Print first 5 records in the DataFrame 'df'
df.head()

In [ ]:
# Print last 5 records in the DataFrame 'df'
df.tail()

Jak můžeme vidět, většina záznamů má autora s přiřazeným kódem, nicméně např. záznam 9608 má autora bez kódu. Nás budou zajímat jen autoři s přiřazeným kódem.  

Abychom nemuseli psát stejný kód několikrát, napíšeme ho jednou do funkce, kterou pak jednoduše zavoláme. V tomto případě si napíšeme funkci, která nám z několika listů vnořených do sebe vytvoří jeden. To se nám bude hodit, až budeme chtít spočítat četnost. 

In [ ]:
# Function that flattens nested lists 
def flatten_list(strings):
    flattened_list = []
    if strings is not None: # Check if element is not None
        for item in strings:
            if isinstance(item, str):  # If element is a string, add it to the list
                flattened_list.append(item)
            elif isinstance(item, list):  # Recursion
                flattened_list.extend(flatten_list(item))
        return flattened_list

print("Function saved")        

### 2. Extrahování a čištění dat

K časopisu je zpravidla připsáno i místo vydání. Pro naše účely nám stačí ale jen název časopisu. Proto pomocí regulárního výrazu  odstraníme místo vydání, které je napsané v kulatých (v bibliografii českého literárního exilu cle), nebo hranatých závorkách (v bibliografii samizdatu).<br>

Nejprve si určíme regulární výraz, který nám najde string před závorkou. Pak ho pomocí lambda funkce najdeme u hodnot ve sloupci `magazine`. Některé hodnoty mají přebytečné mezery před a po stringu. Ty odstraníme pomocí funkce `strip()`. Nakonec všechny vnořené listy odstraníme pomocí napí funkce `flatten_list()`.

In [ ]:
# regex patern, that finds string before column
# r"(.*?)\(" <- round brackets
pattern_magazine = r"(.*?)\("   #r"(.*?)\[" # <- square brackets  

# Save only string before brackets (if they are there)
df['magazine'] = df['magazine'].apply(lambda x: [re.search(pattern_magazine, y).group(1) if re.search(pattern_magazine, y) else y for y in x])

# Remove redundant whitespaces 
df['magazine'] = df['magazine'].apply(lambda x: [y.strip() for y in x])

# Unique magazines
unique_magazine = np.unique(flatten_list(df.magazine))
print("There are " + str(len(unique_magazine)) + " magazines in the database.")
print("All magazines in the database: \n",unique_magazine)



### 3. Zpracování

Abychom mohli vytvořit graf, musíme data v tabulce zpracovat. V grafu chceme zobrazit deset nejčetnějších autorů a k nim přiřadit časopisy, do kterých autoři přispívali. Příkladu vyselektujeme pouze autory, kteří mají přiřazený kód. Je to kvůli tomu, abychom odstranili nežádoucí záznamy, jako jsou například články, které psala celá redakce časopisu. Ty žádné kódy přiřazené nemají. V našem příkladu se zajímáme jen o konkrétní osoby. <br>    

#### 3.1 Nalezení 10 nejčastějších autorů

Nejprve musíme zjistit všechny autory, kteří se v bibliografii objevují. Pomocí funkce `Counter()` spočteme jejich četnosti a vybereme jen 10 nejčastějších.

In [ ]:
author_column = 'author'

# All authors in the column that have a code
all_authors = flatten_list(df[df['author code'] != None][author_column])

# Count number of authors records
counted_authors = Counter(all_authors)

# Number of nodes we want to print
n = 10

# Find the most common authors
most_common_authors = [item[0] for item in counted_authors.most_common(n)]

print("Ten most common authors: \n", most_common_authors)


#### 3.2 Získání indexů nejčastějších autorů

U 10 nejčastějších autorů najdeme indexy řádků, ve kterých se autoři vyskytovali.  K tomu si napíšeme funkci `find_indices(df, column, most_common)`, která v DataFramu `df` najde indexy všech řádků, které mají alespoň jedno jméno z listu `most_common` ve sloupci `column`. <br> 

In [ ]:
def find_indices(df, column, most_common):
    # List of indices
    ind = []
    
    # Iterate through DataFrame df
    for _, row in df.iterrows():
 
        # If value list is not empty, check, if list contains element from list 'most_common'
        if len(row[column]) > 0 :
            
            # If list contains element from list 'most_common', add True. 
            if any(author in row[column] for author in most_common):
                ind.append(True)
            else:
                # Otherwise add False
                ind.append(False)                        
        else:    
            # Value list is empty, add False           
            ind.append(False)               
    return ind            

print("Function saved.")    

Zavoláme naši funkci `find_indices(df, column, most_common)` a zjistíme indexy všech řádků, kde se objevuje nějaký autor z `most_common_authors`. Pak si je vypíšeme.<br>
List indexů je list true/false hodnot, kde true hodnota znamená, že řádek autora obsahujea a false že neobsahuje.<br>
Abychom neměli výpis tak dlouhý, true hodnoty vypíšeme jako čísla řádků, kde se autor objevuje.  

In [ ]:
# Find all indices of rows that contains authors from 'most_common_authors'  
ind = find_indices(df, author_column, most_common_authors)

# Alternative 
# [True if any([True if author in most_common_authors else False for author in author_list]) else False for author_list in df[author_code_column]]

print([i for i, x in enumerate(ind) if x])

#### 3.3 Nalezení časopisů

Určíme si sloupec, který chceme přiřadit. V našem případě je to sloupec `magazine`. Pak už jen pomocí indexů zjistíme, do jakých časopisů nejčastější autoři publikovali.   

In [ ]:
# Select column that we want to print 
column =  'magazine'

# All magazines to which authors from 'most_common_authors' contributed
author_elements = df[ind][column]

# Unique magazines
unique_author_elements = np.unique(flatten_list(author_elements)) 

print("All magazines to which 10 most common authors contributed: \n", unique_author_elements)

<div class='alert alert-block alert-info'>
    <b>Try It!</b> Pomocí toho kódu lze zobrazit i jiné vztahy dvou sloupců, např. kteří autoři psali o jakých osobnostech (figures). K tomu jen změníme údaj proměnné `column`
</div>

#### 3.4 Spočtení vah/četnosti

Ke každé dvojici ('autor', 'časopis') zjistíme četnost. To uděláme tak. že si vytvoříme dictionary `edge_weights`, kde klíče budou tuples ('autor', 'časopis') a hodnoty počet napsaných článků daným autorem do časopisu. 

In [ ]:
# Edge weights. The more articles in the magazine, the heigher weight
# Higher weight will be represented by thicker line 
edge_weights = {}

# Iterate through rows in the DataFrame
for _,row in df.iterrows():
    for element in row[column]:
        for author in row[author_column]:
            # Select 10 most common elements
            if element in unique_author_elements and author in most_common_authors:
                if  (author, element) in edge_weights:
                    # Add edges as keys to dictionary 
                    edge_weights[(author, element)] += 1
                else:
                    edge_weights[(author, element)] = 1
                    
edge_weights                   

### 4. Vytvoření struktury grafu

V následujícím kroku vytvoříme graf, který nakonec vykreslíme. Využijeme k tomu knihovnu `networkx`, kam jen přidáme dvě sady uzlů - autory a časopisy, a k nim vážené hrany. Graf nakonec vykreslíme.     

#### 4.1 Vložení jedné sady uzlů

Nejprve vytvoříme objekt grafu `G` a vložíme do něj jména nejčastějších autorů jako uzle. 

In [ ]:
# Create graph
G = nx.Graph()

# Add elements as graph's nodes
G.add_nodes_from(most_common_authors, bipartite=1)

G.nodes

#### 4.2 Vložení druhé sady uzlů a hran

Jak jsme mohli vidět výše, hran je opravdu mnoho. Abychom graf neměli přehlcený, omezíme počet vykreslených hran na ty nejčetnější. Parametrem `threshold` nastavíme, jaký je potřeba minimální počet článků od jednoho autora pro to, aby se časopis zobrazil. Do našeho grafu `G` pak přidáme uzle a hrany, se vejdou do omezení parametru `threshold`. 

In [ ]:
# Display only magazines that occur multiple times  
threshold = 15

# Magazines to which at least one author contributed <threshold> articles 
left = []

# Iterate through edges and add them to graph
for edge, weight in edge_weights.items():
    if weight>threshold:
        if ~G.has_node(edge[1]):
            left.append(edge[1])
            G.add_node(edge[1], bipartite=0) 
        G.add_edge(edge[0], edge[1], weight=weight)    

print("Graph structure created.")  
G.edges                  

#### 4.3 Vykreslení grafu

Teď už jen pomocí knihovny `matplotlib` graf vykreslíme. Nejprve si připravíme prázdné plátno, kam graf nakreslíme. Pak si určíme šířku hran. Nakonec graf pomocí funkce `draw_networkx()` vykreslíme. 

In [ ]:
pos = nx.bipartite_layout(G, nodes = left)

# Create plain canvas
plt.figure(figsize=(15, 10))

# Edge width
width = 0.1
edge_widths = [width * G[u][v]['weight'] for u, v in G.edges()]

# Draw graph
nx.draw_networkx(G, pos=pos, with_labels=True, node_color='lightblue', node_size=400,
                 width=edge_widths, edge_color='gray', alpha=0.7)
plt.title("Contribution to exile magazines")
plt.axis("off")
plt.savefig("plots/bipartite_graph.svg")
plt.show()

Na grafu vidíme, do kterých časopisů 10 nejčetnějších autorů psalo své příspěvky.<br> 
Zdaleka nejvíce článků napsal Jaroslav Dresler do časopisu Národní politika, pak do časopisu <i>České slovo</i>. Někteří autoři, jako např. Antonín Kratochvil neb Antonín Měšťan, nebyli spjati pouze s jedním časopisem a publikovali své články do vícero časopisů. Naopak Jiří Kovtun publikoval výhradně do časopisu <i>Svědectví</i>, Pavel Řehoř zase do periodika <i>Zpravodaj</i>.<br> 
Musíme mít na paměti, že časopisy jsou omezeny parametrem `threshold` pro čitelnější vykreslení. Pokud bychom chtěli vykreslit všechny časopisy, `threshold` nastavíme na 1. <br>

Například Josef Škvorecký, jak vidíme v následující buňce, napsal příspěvky do 14 časopisů, nicméně se nám v grafu zobrazují jen dva nejčastější –⁠ <i>Západ</i> a <i>Listy</i>. <br>

In [ ]:
# All magazines to which Skvorecky contributed
j_skvorecky_magazines = df[df['author'].apply(lambda x: any(author == 'Škvorecký, Josef' for author in x ))]['magazine']

# Unique magazines
unique_j_skvorecky_magazine = np.unique(flatten_list(j_skvorecky_magazines))

print("Josef Škvorecký contributed to ",len(unique_j_skvorecky_magazine), " magazines." )
print(unique_j_skvorecky_magazine)


<div class='alert alert-block alert-info'>
    <b>Try It!</b>  Pomocí parametru threshold zkuste přidat nebo ubrat některé hrany. <br>
</div>

Bipartitní graf je dobrý nástroj pro zobrazovní vztahů mezi dvěma entitami. V tomto případě jsme zobrazili vztah mezi autory literárních statí a exilových časopisů, do kterých autoři publikovali. <br>

<div class='alert alert-block alert-info'>
    <b>Try It!</b>  Místo časopisů si můžeme zobrazit sloupec 'figures'. Graf nám pak ukáže, kteří autoři psali o kterých osobnostech. Nebo si můžeme zobrazit žánry, které autoři nejčastěji psali. Bibliografii také můžeme na začátku notebooku časově ohraničit.<br>
    Také se nemusíme omezovat na bibliografii českého literárního exilu. 
</div>